# RedisTimeseriesManager Output Formats Demonstration

See an example of supported output formats.

Supported formats are 'list' (python `list`, default format), 'df'(pandas dataframe), 'dict' (dictioanry of lines), 'sets-list' (sets of data including lables and lists), 'sets-df' (sets of data  including labels and dataframes), 'sets-dict' (sets of data including labels and dictionary of lines) and 'raw' which is the raw data read from the timeseries.

In [1]:
# Initialization
from _path import add_root
add_root(up_levels=1)

from src.redis_timeseries_manager import RedisTimeseriesManager

class RedisTimeseriesManagerTests(RedisTimeseriesManager):
    _name = 'test1'
    _lines = ['l1', 'l2']
    _timeframes = {
        'raw': {'retention_secs': 100000}, #unlimited
    }

default_test_settings = {
    'host': 'localhost',
    'port': 6379,
    'db': 0,
    'password': None,
}
try:
    from settings import test_settings
except:
    test_settings = None
test_settings = test_settings or default_test_settings

tl = RedisTimeseriesManagerTests(**test_settings)

In [2]:
# set/reset test data
def reset_test_data():
    # clear previous data
    keys = tl.query_index(return_key_names=True)
    for key in keys[1]:
        tl.client.delete(key)
    # set test data
    data1 = [
        ['btc',100, 1, 2],
        ['btc',101, 3, 4],
        ['btc',102, 5, 6],
        ['eth', 101, 7, 8],
        ['eth', 102, 9, 10],
        ['eth', 103, 11, 12],
        ['eth', 104, 13, 14],
    ]
    tl.insert(
        data=data1,
        c1='crypto',
        c2_position=0,
        create_inplace=True,
        extra_labels={'iscrypto': 'yes'}
    )

In [3]:
# preparing data
reset_test_data()
c1 = 'crypto'
c2 = {
    'iscrypto': 'yes'
}

### Generating read error by not allowing multiple data-points with the same time

Allowing multiple data-points with the same time can possibly results in data ambiguity if not used with consideration

In [4]:
success, result = tl.read(
    c1=c1,
    c2=c2,
)
result

'Inadequate filters: Multiple lines with the same name are returned. This generally happens when label filters are used as classifiers and labels are not enough. Please add all labels to filter out data properly or turn on the `allow_multiple` option'

## Output as `list` format(default)

In [5]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='list'
)
data

[[100.0, 1.0, 2.0],
 [101.0, 3.0, 4.0],
 [101.0, 7.0, 8.0],
 [102.0, 5.0, 6.0],
 [102.0, 9.0, 10.0],
 [103.0, 11.0, 12.0],
 [104.0, 13.0, 14.0]]

In [6]:
tl.get_read_length(data=data, data_type='list')

7

## Output as `df`(pandas dataframe) format

In [7]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='df'
)
data

,time,l1,l2
0,100,1.0,2.0
1,101,3.0,4.0
2,101,7.0,8.0
3,102,5.0,6.0
4,102,9.0,10.0
5,103,11.0,12.0
6,104,13.0,14.0


In [8]:
tl.get_read_length(data=data, data_type='df')

7

## Output as `dict` format

In [9]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='dict'
)
data

{'time': [100, 101, 101, 102, 102, 103, 104],
 'l1': [1.0, 3.0, 7.0, 5.0, 9.0, 11.0, 13.0],
 'l2': [2.0, 4.0, 8.0, 6.0, 10.0, 12.0, 14.0]}

In [10]:
tl.get_read_length(data=data, data_type='dict')

7

## Output as `sets-list` format

In [11]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='sets-list'
)
data

[({'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'btc',
   'timeframe': 'raw'},
  [[100.0, 1.0, 2.0], [101.0, 3.0, 4.0], [102.0, 5.0, 6.0]]),
 ({'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'eth',
   'timeframe': 'raw'},
  [[101.0, 7.0, 8.0],
   [102.0, 9.0, 10.0],
   [103.0, 11.0, 12.0],
   [104.0, 13.0, 14.0]])]

In [12]:
# read_length returns the length of first set when reading as sets
tl.get_read_length(data=data, data_type='sets-list')

3

## Output as `sets-df` format

In [13]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='sets-df'
)
data

[({'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'btc',
   'timeframe': 'raw'},
     time   l1   l2
  0   100  1.0  2.0
  1   101  3.0  4.0
  2   102  5.0  6.0),
 ({'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'eth',
   'timeframe': 'raw'},
     time    l1    l2
  0   101   7.0   8.0
  1   102   9.0  10.0
  2   103  11.0  12.0
  3   104  13.0  14.0)]

In [14]:
# read_length returns the length of first set when reading as sets
tl.get_read_length(data=data, data_type='sets-df')

3

## Output as `sets-dict` format

In [15]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='sets-dict'
)
data

[({'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'btc',
   'timeframe': 'raw'},
  {'time': [100, 101, 102], 'l1': [1.0, 3.0, 5.0], 'l2': [2.0, 4.0, 6.0]}),
 ({'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'eth',
   'timeframe': 'raw'},
  {'time': [101, 102, 103, 104],
   'l1': [7.0, 9.0, 11.0, 13.0],
   'l2': [8.0, 10.0, 12.0, 14.0]})]

In [16]:
# read_length returns the length of first set when reading as sets
tl.get_read_length(data=data, data_type='sets-dict')

3

## Output as `raw` format

In [17]:
success, data = tl.read(
    c1=c1,
    c2=c2,
    allow_multiple=True,
    return_as='raw'
)
data

{'crypto_btc': {'count': 2,
  'labels': {'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'btc',
   'timeframe': 'raw'},
  'lines': {'l1': [(100000, 1.0), (101000, 3.0), (102000, 5.0)],
   'l2': [(100000, 2.0), (101000, 4.0), (102000, 6.0)]}},
 'crypto_eth': {'count': 2,
  'labels': {'iscrypto': 'yes',
   'tl': 'test1',
   'c1': 'crypto',
   'c2': 'eth',
   'timeframe': 'raw'},
  'lines': {'l1': [(101000, 7.0),
    (102000, 9.0),
    (103000, 11.0),
    (104000, 13.0)],
   'l2': [(101000, 8.0), (102000, 10.0), (103000, 12.0), (104000, 14.0)]}}}

In [18]:
# due to efficiency, read_length returns the length of first set when reading as raw
tl.get_read_length(data=data, data_type='raw')

3